In [116]:
%cd ..
%load_ext autoreload
%autoreload 2

/Users
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# 2. Projection Creation Example

This example shows how to create a projection and add coordinates to a dataset.

In [117]:
from rts.db.dao import DataAccessObject
from rts.db_settings import DATABASE_URL
from rts.db.queries import get_library_id_from_name, read_media_by_library_id, create_projection, update_projection, create_map_projection_feature
from rts.io.media import upload_media_files, create_square_atlases
from rts.api.models import Media, Projection, MapProjectionFeatureCreate
from rts.storage.storage import get_supabase_client
import os
import random
from io import BytesIO
from PIL import Image
import pandas as pd
from dotenv import load_dotenv

load_dotenv()
DataAccessObject().connect(DATABASE_URL)

ARCHIVE_BASE_PATH = os.getenv("BASE_PATH")

In [118]:
# Setup before we can create media objects
archive_name =  "rts"
bucket_name =  archive_name
library_id =  get_library_id_from_name(archive_name)

In [119]:
# Retrieve media files for the projection and the atlas
media = read_media_by_library_id(library_id, 'image', 'thumbnail')
print(f"Found {len(media)} media files for library {library_id}")

Found 99 media files for library 1


In [120]:
# Retrieve and generate all the data we need for the projection and atlas
data = {
    'images': [],
    'coordinates': [],
}

for m in media:
    result = get_supabase_client().storage.from_(bucket_name).download(m['media_path'])
    coordinates = [random.randint(0, 100), random.randint(0, 100), random.randint(0, 100)]
    data['coordinates'].append(coordinates)
    data['images'].append(Image.open(BytesIO(result)))

In [121]:
# In this example we want to split the media files into multiple atlases
atlas_count = 2
total_tiles = len(media)
atlas_width = 1024
atlas_tile_size = 128

projection = Projection(
    projection_name="rts_random",
    version="0.0.1",
    library_id=library_id,
    model_name="random",
    model_params={},
    data={},
    dimension=3,
    atlas_folder_path="",
    atlas_width=atlas_width,
    tile_size=atlas_tile_size,
    atlas_count=atlas_count,
    total_tiles=total_tiles,
    tiles_per_atlas=64,
)
projection_id = create_projection(projection)['projection_id']

In [122]:
square_atlases = create_square_atlases("atlas_rts_001", projection_id, data['images'], width=atlas_width)

In [123]:
projection.tile_size = square_atlases[0]['tile_size'][0]
projection.atlas_count = len(square_atlases)
projection.total_tiles = total_tiles
projection.tiles_per_atlas = square_atlases[0]['image_count']

update_projection(projection_id, projection)

In [110]:
media[8]

{'media_id': 42, 'media_path': 'rts/images/ZB006020/256px/ZB006020-L000-02.jpg', 'original_path': '/Users/arattinger/Projects/rts/rts/testdata/data/ZB006020/clips/images/256px/ZB006020-L000-02.jpg', 'original_id': 'ZB006020', 'created_at': datetime.datetime(2023, 6, 19, 15, 13, 4, 403027), 'media_type': 'image', 'file_id': 'ZB006020-L000-02', 'sub_type': 'thumbnail', 'size': 8825, 'metadata': {}, 'library_id': 1, 'hash': '6b05dd0c74d7af795ed0cd945add582a', 'parent_id': 3, 'start_ts': 0.0, 'end_ts': 10.0, 'start_frame': 0, 'end_frame': 10, 'frame_rate': 30.0}

In [126]:
# for m in media:
count = 0
for atlas_order, key in enumerate(square_atlases):
    # print(len(square_atlases[key]['images']))
    for i in range(len(square_atlases[key]['images'])):
        create_map_projection_feature(
            MapProjectionFeatureCreate(
                projection_id=projection_id,
                media_id=media[count]['media_id'],
                atlas_order=atlas_order + 1,
                coordinates=[random.randint(0, 100), random.randint(0, 100), random.randint(0, 100)],
                index_in_atlas=i,
            )
        )
        # print(i, count, atlas_order + 1)
        count += 1